# [LVV-T2214] - MTMount_Elevation_Changes_with_MTAOS_Aberrations

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

## Requirements

This notebooks require some extra repositories to be installed locally so it can grab some constants and some look-up tables.  
Here is a list of which repos are these:

* [lsst-ts/ts_cRIOpy]

[lsst-ts/ts_cRIOpy]: https://github.com/lsst-ts/ts_cRIOpy 
[lsst-ts/ts_m1m3support]: https://github.com/lsst-ts/ts_m1m3support

To install them, open a terminal and run the following commands:

```
git clone https://github.com/lsst-ts/ts_cRIOpy $HOME/notebooks/lsst-ts/ts_cRIOpy
git clone https://github.com/lsst-ts/ts_m1m3support.git $HOME/notebooks/lsst-ts/ts_m1m3support
git clone https://github.com/lsst-ts/ts_config_mttcs $HOME/notebooks/lsst-ts/ts_config_mttcs
git clone https://github.com/lsst-sitcom/M2_FEA $HOME/notebooks/lsst-sitcom/M2_FEA
```

And add these lines to your `$HOME/notebooks/.user_setup` file: 

```
setup -j notebooks_vandv -r ~/notebooks/lsst-sitcom/notebooks_vandv
setup -j ts_cRIOpy -r $HOME/notebooks/lsst-ts/ts_cRIOpy
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [1]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings

from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

In [60]:
test_execution = "LVV-D220706"

In [5]:
log = logging.getLogger("LVV-T2214")
log.setLevel(logging.DEBUG)

In [6]:
lsst_efd_client.EfdClient.list_efd_names()

['ldf_int_efd',
 'base_efd',
 'tucson_teststand_efd',
 'test_efd',
 'summit_efd',
 'ncsa_teststand_efd',
 'ldf_stable_efd']

In [7]:
#efd_name = "summit_efd"
location = "summit"

if location == "summit":
    client = lsst_efd_client.EfdClient("summit_efd")
elif location == "tucson":
    client = lsst_efd_client.EfdClient("tucson_teststand_efd")
else:
    raise ValueError(
        "Location does not match any valid options {summit|tucson}"
        )

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID"))

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
warnings.filterwarnings("ignore", message=".*DDS read queue.*")

## Helper Functions

In [ ]:
async def get_data_from_efd(location, start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    location : str
        In which Test Stand are you running this test?
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """
    if location == "summit":
        client = EfdClient("summit_efd")
    elif location == "tucson":
        client = EfdClient("tucson_teststand_efd")
    else:
        raise ValueError(
            "Location does not match any valid options {summit|tucson}"
        )
    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start, 
        end=end,
    )
    
    _df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="zForce101",
        start=start, 
        end=end,
    )
        
    # M2 B1?
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
            "lutGravity0",
            "measured0"
        ],
        start=start, 
        end=end,
    )
    
    # CamHex Y position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=1,
        start=start, 
        end=end,
    )
    
    # M2Hex Y position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=2,
        start=start, 
        end=end,
    )

    # Rename columns
    _df_mount_el.rename(columns={"actualPosition": "mount_el"}, inplace=True)
    _df_m1m3_101.rename(columns={"zForce101": "m1m3_z101"}, inplace=True)
    _df_m2.rename(columns={"applied0": "m2b1_applied", "lutGravity0": "m2b1_gravLut", "measured0": "m2b1_measured"}, inplace=True)
    _df_camhex.rename(columns={"position1": "camhex_y"}, inplace=True)
    _df_m2hex.rename(columns={"position1": "m2hex_y"}, inplace=True)

    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3_101, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
         
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

In [ ]:
async def moveMountInElevationSteps(target_el, azimuth=0, step_size=0.25, time_sleep=1):
    """Move the mount from the current elevation angle to the target elevation angle 
    in steps to avoid any issues whe M1M3 and/or M2 are running with the LUT using the 
    Mount instead of the inclinometer.
    
    This function will actually calculate the number of steps using the ceiling
    in order to make sure that we move carefully. 
    
    Parameters
    ----------
    target_el : float
        Target elevation angle in degrees
    azimuth : float
        Azimuth angle in degres (default)
    step_size : float
        Step elevation size in degrees (default: 0.25)
    time_sleep : float
        Sleep time between movements (default: 1)
        
    Returns
    -------
    azimuth : float
        Current azimuth
    elevation : float
        Current elevation
    """
    current_el = mtcs.rem.mtmount.tel_elevation.get().actualPosition
    n_steps = int(np.ceil(np.abs(current_el - target_el) / step_size))

    for el in np.linspace(current_el, target_el, n_steps):
        print(f"Moving elevation to {el:.2f} deg")
        await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=azimuth, elevation=el)
        time.sleep(time_sleep)
        
    return azimuth, el

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount
- [ ] MTRotator
- [ ] MTPtg

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Check elevation motion is smooth and no abrupt changes occur (Otherwise it faults M1M3) 

Do the following slew sequence, watch chronograph to make sure there is no abrupt change in elevation angle (otherwise it faults M1M3).

Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

Move to zenith at the end (so that we can start m1m3 with LUT in mount telemetry mode)

In [ ]:
# Move to zenith position and azimuth of 190.5 deg. 
warnings.simplefilter("ignore")
await moveMountInElevationSteps(90., azimuth=190.5)

In [ ]:
# We start tracking at 85.4 deg because the track fails at 85.5 deg

target_elevations = [85.4, 84.4, 83.4, 82.4]

t_start = Time.now()
t_start.format = "isot"
print(f"Checking Elevation Motion - Start time: {t_start}")

for _el in target_elevations:
    time.sleep(2.)
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end = Time.now()
t_end.format = "isot"
print(f"Checking Elevation Motion - End time: {t_end}")

In [ ]:
# Moving to zenith
await moveMountInElevationSteps(89.5, azimuth=190.5)

Plot elevation 

In [ ]:
# Query telemetry
df = await get_data_from_efd(
    exec_info.loc, 
    t_start, 
    t_end)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10))

axs.plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs.set_ylabel("Mount El\n[deg]")

axs.grid(":", alpha=0.5)
axs.legend()

fig.suptitle(f"{test_execution} - Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_el_vs_time_without_aberr.png")
plt.show()

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

---
## Gather Data - Without Aberrations
Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

Added an additional time.sleep(2.) to allow direct comparison with the data with added aberrations. 

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]

t_start_without_aberration = Time.now()
t_start_without_aberration.format = "isot"
print(f"Gathering data - without Aberrations - Start time: {t_start_without_aberration}")

for _el in target_elevations:
    time.sleep(2.)
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_without_aberration = Time.now()
t_end_without_aberration.format = "isot"
print(f"Gathering data - without Aberrations - End time: {t_end_without_aberration}")

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
print(f"Gathering data - without Aberrations - Start time: {t_start_without_aberration}")
print(f"Gathering data - without Aberrations - End time: {t_end_without_aberration}")

In [ ]:
# Query telemetry
df = await get_data_from_efd(
    exec_info.loc, 
    t_start_without_aberration, 
    t_end_without_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["m1m3_z101"].dropna(), "C0o-", label="z101")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df["m2b1_applied"].dropna(), "C1-", label="applied")
# axs[2].plot(df["m2b1_gravLut"].dropna(), "C2-", label="Gravity LUT")
axs[2].plot(df["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_without_aberr.png")
plt.show()

---
## Gather Data - With Aberrations
Do 4 slews in sequence.
For each slew,

2 seconds before the slew starts, use MTAOS to add some aberrations to the system. The aberrations should correspond to 1um of z4 (focus), z5(astigmatism), z7 (y-coma), and z9 (trefoil), respectively track for 39s, simulating a visit.

Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [1]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_aberration = Time.now()
t_start_with_aberration.format = "isot"
print(f"Gathering data - with Aberrations (reset every time) - Start time: {t_start_with_aberration}")

for _el, _ab in zip(target_elevations, aberrations):
    
    print(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    
t_end_with_aberration = Time.now()
t_end_with_aberration.format = "isot"
print(f"Gathering data - with Aberrations (reset every time) - End time: {t_end_with_aberration}")

NameError: name 'mtcs' is not defined

In [ ]:
print(t_start_with_aberration, t_end_with_aberration)

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
# Query telemetry
df_aberr = await get_data_from_efd(
    exec_info.loc, 
    t_start_with_aberration, 
    t_end_with_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df_aberr["m2b1_applied"].dropna(), "C1^-", label="applied")
# axs[2].plot(df_aberr["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].plot(df_aberr["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df_aberr["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation with Aberrations vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

## Gather Data - With Accumulated Aberrations

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_acc_aberration = Time.now()
t_start_with_acc_aberration.format = "isot"
print(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_acc_aberration}")

for _el, _ab in zip(target_elevations, aberrations):
    
    print(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await moveMountInElevationSteps(_el, azimuth=190.5)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_with_acc_aberration = Time.now()
t_end_with_acc_aberration.format = "isot"
print(f"Gathering data - with Aberrations Accumulated - End time: {t_start_with_acc_aberration}")

In [ ]:
print(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_acc_aberration}")
print(f"Gathering data - with Aberrations Accumulated - End time: {t_end_with_acc_aberration}")

### Plot Optics vs Time

In [ ]:
# Query telemetry
df_aberr_acc = await get_data_from_efd(
    exec_info.loc, 
    t_start_with_acc_aberration, 
    t_end_with_acc_aberration)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr_acc["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr_acc["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

# axs[2].plot(df_aberr_acc["m2b1_applied"].dropna(), "C1^-", label="applied")
# axs[2].plot(df_aberr_acc["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].plot(df_aberr_acc["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df_aberr_acc["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr_acc["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation with Aberrations vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

In [ ]:
print( mtcs.rem.mtaos.evt_m2Correction.get() )

In [ ]:
print( mtcs.rem.mtaos.evt_m2Correction.get() )
await mtcs.rem.mtaos.cmd_resetCorrection.set_start()
print( mtcs.rem.mtaos.evt_m2Correction.get() )

w = np.zeros(19)
w[0] = 1

await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=w)
print( mtcs.rem.mtaos.evt_m2Correction.get() )

## Compare against the corrections sent by MTAOS. 

This is done in a separate notebook, LVV-T2214-plots.

## Wrap up
lower m1m3,
put hardware to OFFLINE
put all simulators to STANDBY
done.

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
await mtcs.point_azel(az=0, el=80)

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )